In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive

Mounted at /content/gdrive
MyDrive


In [ ]:
!curl ipecho.net/plain

34.82.132.196

In [ ]:
%cd /content/gdrive/My\ Drive/Thesis/MovieLens20M

/content/gdrive/My Drive/Thesis/MovieLens20M


In [ ]:
!pip install tqdm

In [ ]:
!pip install scenedetect[opencv]

     |████████████████████████████████| 98 kB 4.5 MB/s 


In [ ]:
!pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=d25b42c00c9d5e1b91e793672a1ddf1774aa6cf77004826112a549d48a4bd615
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [ ]:
!pip install -U kora

     |████████████████████████████████| 57 kB 3.0 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 


In [ ]:
import glob
import os
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.editor import *
import numpy as np
from kora.drive import upload_public
from IPython.core.display import display, HTML
from base64 import b64encode
from IPython.display import Image, display
import shutil
from __future__ import print_function
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors.content_detector import ContentDetector
from tqdm import tqdm

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3096576/45929032 bytes (6.7%)6397952/45929032 bytes (13.9%)9658368/45929032 bytes (21.0%)13287424/45929032 bytes (28.9%)16990208/45929032 bytes (37.0%)20635648/45929032 bytes (44.9%)24150016/45929032 bytes (52.6%)27557888/45929032 bytes (60.0%)30916608/45929032 bytes (67.3%)33693696/45929032 bytes (73.4%)36429824/45929032 bytes (79.3%)39264256/45929032 bytes (85.5%)

#Loaders

In [ ]:
def load_name(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['ID_Frame']
    return (df.loc[ids == id])['Name'].values.tolist()
def load_label_old(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['movieId']
    genres_str = (df.loc[ids == id])['genres'].values.tolist()
    if genres_str:
        genres = genres_str[0].split('|')
        return genres
    else:
        return []
def load_label(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['ID_Frame']
    genres_str = (df.loc[ids == id])['Genres'].values.tolist()
    if genres_str:
        genres = genres_str[0].split('|')
        return genres
    else:
        return []
def load_label2(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['ID_Frame']
    genres_str = (df.loc[ids == id])['Genres_subset'].values.tolist()
    if genres_str:
        genres = genres_str[0].split('|')
        return genres
    else:
        return []

#Fill CSV from downloaded data - or update existing CSV

In [ ]:
def fill_csv(images_path,csv_path,main_genre,csv_out_name=None,reset=False):
    directory = os.listdir(images_path)
    
    try:
        df= pd.read_csv(csv_path)
        out_df = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres'])
        dataframe.to_csv(csv_out_name)
        out_df = pd.read_csv(csv_out_name)
        df=pd.read_csv(csv_path)

    
    #Drop unnamed ranges.
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    if out_df is not None:
        print("Using existing csv...")
    for img_name in tqdm(directory):
        try:
            id = int(img_name.split('.')[0])
            if not img_name in out_df['ID_Frame']:
                new_row = {'ID_Frame':id,'Genres': load_label_old(csv_path,id)}
                out_df = out_df.append(new_row, ignore_index=True)

                out_df.to_csv(csv_out_name,index=False)
        except:
            print('skipped' , img_name)


In [ ]:

main_genre = ['Adventure', 'Comedy', 'Fantasy', 'Animation-Children', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'Documentary', 'Musical']

fill_csv('Data',"CsvFiles/movies.csv",main_genre,"CsvFiles/movies_current.csv")

Using existing csv...


  3%|▎         | 226/6681 [00:10<04:37, 23.30it/s]

skipped 10000Hz.mp4.part


100%|██████████| 6681/6681 [06:42<00:00, 16.60it/s]


#Show video

In [ ]:
def show_video(name):
    mp4 = open(name,'rb').read()
    decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
            
    display (HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % decoded_vid) )
def show_video_2 (video_name):
    url = upload_public(video_name)
    
    display(HTML(f"""<video src={url} width=500 controls/>"""))

def show_all_videos(path):
    directory= os.listdir(path)
    for video in directory:
        print(os.path.join(path,video))
        url = upload_public(os.path.join(path,video))
        display(HTML(f"""<video src={url} width=500 controls/>"""))


#Find Scenes

In [ ]:


def find_scenes(video_path):
    # type: (str) -> List[Tuple[FrameTimecode, FrameTimecode]]
    video_manager = VideoManager([video_path])
    print(video_manager)
    stats_manager = StatsManager()
    # Construct our SceneManager and pass it our StatsManager.
    scene_manager = SceneManager(stats_manager)
    
    # Add ContentDetector algorithm (each detector's constructor
    # takes detector options, e.g. threshold).
    
    scene_manager.add_detector(ContentDetector(threshold=40.0, min_scene_len=1))
    base_timecode = video_manager.get_base_timecode()

    # We save our stats file to {VIDEO_PATH}.stats.csv.
    stats_file_path = '%s.stats.csv' % video_path

    scene_list = []

    try:
        # If stats file exists, load it.
        if os.path.exists(stats_file_path):
            # Read stats from CSV file opened in read mode:
            with open(stats_file_path, 'r') as stats_file:
                stats_manager.load_from_csv(stats_file, base_timecode)

        # Set downscale factor to improve processing speed.
        video_manager.set_downscale_factor()

        # Start video_manager.
        video_manager.start()
        try: 
            # Perform scene detection on video_manager.
            scene_manager.detect_scenes(frame_source=video_manager)

            # Obtain list of detected scenes.
            scene_list = scene_manager.get_scene_list(base_timecode)
            # Each scene is a tuple of (start, end) FrameTimecodes.

            #print('List of scenes obtained:')
            #for i, scene in enumerate(scene_list):
            #   print(
            #        'Scene %2d: Start %s / Frame %d, End %s / Frame %d' % (
            #        i+1,
            #       scene[0].get_timecode(), scene[0].get_frames(),
              #      scene[1].get_timecode(), scene[1].get_frames(),))

            # We only write to the stats file if a save is required:
            if stats_manager.is_save_required():
                with open(stats_file_path, 'w') as stats_file:
                    stats_manager.save_to_csv(stats_file, base_timecode)
        finally:
            video_manager.release()

    finally:
        video_manager.release()

    return scene_list
     

In [ ]:
a= find_scenes('Data/2.mp4')

NameError: ignored

#Clean Splits

In [ ]:
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors.content_detector import ContentDetector
from scenedetect.detectors.threshold_detector import ThresholdDetector
def  clean_splits(cut_list):
    #Clean the splits.
    if len(cut_list)<3:
        return 
    cut_list = cut_list[1:-1]
    min_threshold =  (int(cut_list[-1][-1])- int(cut_list[0][0]))  / 60
    max_threshold = (int(cut_list[-1][-1])- int(cut_list[0][0]))  / 10
    new_cut_list = []
    start_t = cut_list[0][0]
    for (start_time,end_time) in cut_list:
        dur = int(end_time)-int(start_t)
        dur2 = int(end_time)- int(start_time)
        
        if dur2 > min_threshold and dur2 < max_threshold:
            dur3  = int(start_time) - int(start_t)
            if dur3 > min_threshold and dur3 < max_threshold:
                new_cut_list.append((start_t,start_time))
                new_cut_list.append((start_time,end_time))
                start_t = end_time
            else:
                start_t = end_time
                new_cut_list.append((start_time,end_time))
        elif dur > min_threshold and dur < max_threshold:
            new_cut_list.append((start_t,end_time))
            start_t = end_time

    return new_cut_list

     

In [ ]:
print(len(a))
print(len(clean_splits(a)))

In [ ]:

def cut_frames(video_path,destination,clean_split=False,show_results=False,reset=False):
    print(video_path)
    print(destination)
    directory = video_path.split('/')[0]
    video_id = (video_path.split('/')[1]).split('.')[0]
    if len((video_path.split('/')[1]).split('.')) > 2:
        return
    current_path = os.path.join(directory, str(video_id))

  
    #If scene method is 1 or 2. you can give threshold with method 2. Method 1
    #Finds optimal threshold in its own way. Scene 1 is faster..
    cut_list = find_scenes(video_path) 
    if cut_list is type(None):
        print("An error appeared with find scenes... returning the function..")
        return
    #If it is going to clean the splits:
    if clean_split:
        new_cut_list=clean_splits(cut_list)
    else:
        new_cut_list=cut_list

    if new_cut_list:    
        print("I will make ", len(new_cut_list), " cuts")
        if len(new_cut_list)< 1 :
            print("I couldn't make any cuts. (<1)")
            return
        for indx,(start_time,end_time) in enumerate(new_cut_list):     
            target = os.path.join(destination,video_id+'_'+str(indx)+'.png')
            print("Saving frame to the target... " + target)
          
            #ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=target)
            try:
                clip = VideoFileClip(video_path) 
            except:
                print("Error in the VideoFileClip!")
                return
            clip.save_frame(target, t = (start_time.get_seconds()+ end_time.get_seconds())/2) 

        if show_results:
            show_all_images(current_path)

    else:
        print("Error finding cut list. (Nonetype). id : " , video_id)

In [ ]:
def cut_clips(video_path,clean_split=False, show_results=True):

    directory = video_path.split('/')[0]
    video_id = video_path.split('/')[1][:-4]
    
    #If scene method is 1 or 2. you can give threshold with method 2. Method 1
    #Finds optimal threshold in its own way.
    
    cut_list = find_scenes(data)
    current_path = os.path.join(directory, str(video_id))
   
    #If it is going to clean the splits:
    if clean_split:
        new_cut_list=clean_splits(cut_list)
    else:
        new_cut_list=cut_list

    try:
        print("Saving to the path :" , current_path)
        os.mkdir(current_path)
    except FileExistsError:
        print("Already existing directory")


    print("I will make ", len(new_cut_list), " cuts")
    for indx,(start_time,end_time) in enumerate(new_cut_list):     
        target = os.path.join(current_path, str(indx)+'.mp4')
        print("Saving video to the target... " + target)
        if not len(os.listdir(current_path)) == len(new_cut_list):
            ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=target)

    if show_results:
        show_all_videos(current_path)


# New code to insert into csv


In [ ]:
def csv_clips(video_path, clean_split=False, show_results=True):

    directory = video_path.split('/')[0]
    video_id = video_path.split('/')[1][:-4]
    #If scene method is 1 or 2. you can give threshold with method 2. Method 1
    #Finds optimal threshold in its own way.
    
    cut_list = find_scenes(data)

    #If it is going to clean the splits:
    if clean_split:
        new_cut_list=clean_splits(cut_list)
    else:
        new_cut_list=cut_list

    for indx,(start_time,end_time) in enumerate(new_cut_list):     
        
        print("Saving video to the target... " + target)
    



In [ ]:
def find_all_scenes_to_csv(data_folder,csv_in_name,csv_out_name):
    csv_in = pd.read_csv(csv_in_name)
    
    #Drop unnamed ranges.
    csv_in.drop(csv_in.columns[csv_in.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    files= csv_in['ID_Frame'].tolist()
    try:
        csv_out = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points'])
        dataframe.to_csv(csv_out_name)
        csv_out = pd.read_csv(csv_out_name)
        df=pd.read_csv(csv_in_name)
    ids = csv_out['ID_Frame'].tolist()

    files = [str(f) + '.mp4' for f in files]
    for i,data in enumerate(files):
        
        if ".mp4" in data and ".csv" not in data:
            video_id = int(data.split('.')[0])
            print(video_id)
            if video_id in ids:
                print("Skipping video_id", video_id)
            else:
                current_path = os.path.join(data_folder,data)
                print("Doing video " ,current_path)
                if os.path.exists(current_path):
                    splits = clean_splits(find_scenes(current_path))
                    if splits:
                        splits_list =  [x[0].get_frames() for x in splits]
                        print("Saving splits to csv", splits_list)
                        new_row = {
                        'ID_Frame':int(video_id),
                        
                        'Genres': load_label(csv_in_name,int(video_id)),
                        'Cut_Points': splits_list,
                        'Number_Frames': len(splits_list)
                        }
                      
                        csv_out = csv_out.append(new_row, ignore_index=True)
                        csv_out.to_csv(csv_out_name,index=False)
                    else:
                        print("could not split video with id:", current_path)
                    
    print("Done.")
  
    

In [ ]:

find_all_scenes_to_csv('Data', "CsvFiles/movies_current.csv", "CsvFiles/movies_current_scenes.csv")

1792131
Doing video  Data/1792131.mp4
1792132
Doing video  Data/1792132.mp4
1792133
Doing video  Data/1792133.mp4
1792134
Doing video  Data/1792134.mp4
1792135
Doing video  Data/1792135.mp4
1792136
Doing video  Data/1792136.mp4
1792137
Doing video  Data/1792137.mp4
1792138
Doing video  Data/1792138.mp4
1792139
Doing video  Data/1792139.mp4
1792140
Doing video  Data/1792140.mp4
1792141
Doing video  Data/1792141.mp4
1792142
Doing video  Data/1792142.mp4
1792143
Doing video  Data/1792143.mp4
1792144
Doing video  Data/1792144.mp4
1792145
Doing video  Data/1792145.mp4
1792146
Doing video  Data/1792146.mp4
1792147
Doing video  Data/1792147.mp4
1792148
Doing video  Data/1792148.mp4
1792149
Doing video  Data/1792149.mp4
1792150
Doing video  Data/1792150.mp4
1792151
Doing video  Data/1792151.mp4
1792152
Doing video  Data/1792152.mp4
1792153
Doing video  Data/1792153.mp4
1792154
Doing video  Data/1792154.mp4
1792155
Doing video  Data/1792155.mp4
1792156
Doing video  Data/1792156.mp4
1792157
Doin

100%|██████████| 3499/3499 [00:08<00:00, 415.73frames/s]


could not split video with id: Data/25907.mp4
26338
Doing video  Data/26338.mp4


100%|██████████| 7309/7309 [00:42<00:00, 172.12frames/s]


could not split video with id: Data/26338.mp4
7771
Doing video  Data/7771.mp4


...

100%|██████████| 3641/3641 [00:06<00:00, 568.22frames/s]


Saving splits to csv [1, 105, 345, 417, 567, 647, 717, 789, 848, 911, 983, 1061, 1156, 1220, 1279, 1372, 1431, 1563, 1627, 1692, 1764, 1847, 1907, 1993, 2055, 2117, 2179, 2243, 2337, 2476, 2556, 2615, 2679, 2755, 2819, 2897, 2960, 3027, 3095, 3155, 3243, 3320, 3403]
5480
Doing video  Data/5480.mp4


100%|██████████| 3514/3514 [00:09<00:00, 359.31frames/s]


Saving splits to csv [26, 102, 182, 272, 330, 410, 493, 572, 648, 708, 773, 858, 916, 984, 1070, 1132, 1252, 1343, 1458, 1532, 1636, 1720, 1790, 1888, 1984, 2057, 2131, 2203, 2289, 2358, 2478, 2543, 2616, 2676, 2748, 2817, 2883, 2958, 3024, 3097, 3168, 3237, 3295, 3372]
380
Doing video  Data/380.mp4


 80%|████████  | 2833/3525 [00:08<00:02, 322.46frames/s]

#Show images in a path

In [ ]:
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
def show_all_images(path):
    fig = figure(dpi=200, facecolor='w', edgecolor='k')
    
    directory= os.listdir(path)
    number_of_files = len(directory)

    for i,img in enumerate(directory):
        #print(os.path.join(path,img))
        
        a=fig.add_subplot(1,number_of_files,i+1)
        image = imread(os.path.join(path,img))
        imshow(image,cmap='Greys_r')
        axis('off')
    return


#Clean up for csv 

In [ ]:
def clean_up(folder_name):
    directory= os.listdir(folder_name)
    for f in directory:
      current_path = os.path.join(folder_name,f)
      if 'csv' in f or 'stats' in f:
          print("Removing .csv")
          print(current_path)
          os.remove(current_path)
      #elif 'mp4' in f and len(f)>8:
      #    print("Removing some videos")
      #    print(current_path)
      #    os.remove(current_path)
      if os.path.isdir(current_path):
          if len(os.listdir(current_path)) < 2:
              print("Removing empty folder")
              print(current_path)
              shutil.rmtree(current_path)
          else:
              print("Removing folders")
              print(current_path)
              shutil.rmtree(current_path)



In [ ]:
clean_up("Data")

In [ ]:
def find_all_scenes(data_folder,destination,statistics):
    stats = pd.read_csv(statistics)
    files = os.listdir(folder_name)
    for i,data in enumerate(files):
        
        if ".mp4" in data and ".csv" not in data:
            video_id = data.split('.')[0]
            print(video_id)
            if video_id in stats.ID_Frame:
                print("Skipping video_id", id)
            else:
                current_path = os.path.join(data_folder,data)
                
                cut_frames(current_path,destination,clean_split=True,show_results=False)
                print("Saving image to ", destination)
                
    print("Done.")
  
    

In [ ]:
from tqdm import tqdm
from keras.preprocessing import image
import pickle
def construct_video_trainingds():

    all_frames = pd.read_csv('trial.csv')
    all_frames["Parent"] = all_frames["ID_Frame"].apply(lambda x:x.split('_')[0])
    all_frames.head()
    #print(all_frames)
    stats = pd.read_csv("statistics.csv")
    # dropping duplicate values 
    all_frames.drop_duplicates(keep='last',inplace=True) 
    all_frames.to_csv('trial.csv')
    # for loop to read and store frames
    for i in tqdm(range(stats.shape[0])):
        # creating an empty list
        train_image = []
        # loading the image and keeping the target size as (224,224,3)
        frames = all_frames[all_frames['Parent'].astype(int) == stats['ID_Frame'][i].astype(int)]
        if len(frames )!=stats['# of Frames'][i]:
            print("Oh no! Problem with csvs.")
        for f in range(int(stats['# of Frames'][i])):
            img = image.load_img('VisualDataset/'+stats['ID_Frame'][i].astype(str)+"_"+str(f)+".png", target_size=(224,224,3))
            print(img)
            # converting it to array
            img = image.img_to_array(img)
            # normalizing the pixel value
            img = img/255
            # appending the image to the train_image list
            train_image.append(img)
        return

In [ ]:

def fill_csv(images_path,csv_in_name,csv_out_name,main_genre,reset=False):
    directory = os.listdir(images_path)
    im = [l for l in directory if '.png'  in l]
    images = [i.split('_', 1)[0] for i in im]
    images_set = set(images)
    print("total count of data is" , len(images))
    try:
        df= pd.read_csv(csv_in_name)
        out_df = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres'])
        dataframe.to_csv(csv_out_name)
        out_df = pd.read_csv(csv_out_name)
        df=pd.read_csv(csv_in_name)
    if out_df is not None:
        print("Using existing csv...")
    ids = out_df['ID_Frame'].to_numpy()
    print(ids)
    for id in images_set:
        print("Doing file with id ", id)
        
        if  np.int64(id) !=ids:
            
            frames = [j for j in images if id in j]
            new_row = {
                'ID_Frame':int(id),
                'Name': load_name(csv_in_name,int(id)),
                '# of Frames': len(frames),
                'Genres': load_label(csv_in_name,int(id)),
        
              }
            out_df = out_df.append(new_row, ignore_index=True)
      
            out_df.to_csv(csv_out_name,index=False)
        else:
            print("skipped.")
  
    print("Done.")

In [ ]:
clean_up("Data")

#Loaders 2


In [ ]:
#Delete cut points with <50 frames.
def load_frames_list_old(df,id):

    ids = df['ID_Frame'].values.astype(int)
    if  (df.loc[ids == id])['Number_Frames'].values.astype(int)>0:
        return (df.loc[ids == id])['Cut_Points'].apply(lambda x:   [list(map(int, a.split(","))) for a in  np.array(x.replace(' ','').strip(']][[').split("],["))]).tolist()[0]

    else:
        return []
def load_frames_list(df,id):

    ids = df['ID_Frame'].values.astype(int)
    if  (df.loc[ids == id])['Number_Scenes'].values.astype(int)>0:
        return (df.loc[ids == id])['Cut_Points'].apply(lambda x:   [list(map(int, a.split(","))) for a in  np.array(x.replace(' ','').strip(']][[').split("],["))]).tolist()[0]

    else:
        return []

def load_name(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['ID_Frame'].values.astype(int)
    return (df.loc[ids == id])['Name'].astype(str)
def load_label(df,id):
    ids = df['ID_Frame'].values.astype(int)
    genres_str = (df.loc[ids == id])['Genres'].values.tolist()
    if genres_str:
        genres = genres_str[0].strip('\"[\'\'\']\"').split('|')
        return genres
    else:
        return []
def load_label2(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['ID_Frame'].values.astype(int)
    genres_str = (df.loc[ids == id])['Genres_subset'].values.tolist()
    if genres_str:
        genres = genres_str[0].split('|')
        return genres
    else:
        return []

#Preprocess the cut points.


In [ ]:
!pip install scenedetect[opencv]

In [ ]:

def clean_cut_points(csv_in_name,csv_out_name, delete_limit_under=0, delete_limit_upper= 10000):
    csv_in = pd.read_csv(csv_in_name, engine='python')
    
    files= csv_in['ID_Frame'].tolist()
    try:
        csv_out = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points','Number_Scenes'])
        dataframe.to_csv(csv_out_name)
        csv_out = pd.read_csv(csv_out_name)
    skip_ids = csv_out['ID_Frame'].tolist()
    ids = csv_in['ID_Frame'].tolist()
    
    for i,data in tqdm(enumerate(ids)):
        cut_points = load_frames_list_old(csv_in,data)[0]
        
        if data not in skip_ids:
            valid_cut_points = [[cut_points[i],cut_points[i+1]] for i,b in enumerate(cut_points[:-1]) 
                if cut_points[i+1]-cut_points[i] > delete_limit_under and 
                cut_points[i+1]-cut_points[i] < delete_limit_upper]
            if len(valid_cut_points)>=1 and load_label(csv_in,int(data)):
                new_row = {
                            'ID_Frame':int(data),
                            'Genres': load_label(csv_in,int(data)),
                            'Cut_Points': valid_cut_points,
                            'Number_Scenes': len(valid_cut_points)
                            }

            
                csv_out = csv_out.append(new_row, ignore_index=True)
                csv_out.to_csv(csv_out_name,index=False)

#Clean only empty labels or 0 length scene datas
def clean_cut_points2(csv_in_name,csv_out_name):
    csv_in = pd.read_csv(csv_in_name)
    
    files= csv_in['ID_Frame'].tolist()
    try:
        csv_out = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points','Number_Scenes'])
        dataframe.to_csv(csv_out_name)
        csv_out = pd.read_csv(csv_out_name)
    skip_ids = csv_out['ID_Frame'].tolist()
    ids = csv_in['ID_Frame'].tolist()
    
    for i,data in tqdm(enumerate(ids)):
        cut_points = load_frames_list(csv_in,data)
        if data not in skip_ids:
            
            if len(cut_points)> 0 and load_label(csv_in,int(data)):
                new_row = {
                            'ID_Frame':int(data),
                            'Genres': load_label(csv_in,int(data)),
                            'Cut_Points': cut_points,
                            'Number_Scenes':len(cut_points)
                            }

            
                csv_out = csv_out.append(new_row, ignore_index=True)
                csv_out.to_csv(csv_out_name,index=False)

In [ ]:
import os
os.remove("CsvFiles/movies_current_cleaned.csv")
clean_cut_points( "CsvFiles/movies_current_scenes.csv", "CsvFiles/movies_current_cleaned.csv",100,1800) 


5675it [04:34, 20.65it/s]


In [ ]:
import os
os.remove("CsvFiles/movies_current_cleaned_2.csv")
clean_cut_points2("CsvFiles/movies_current_cleaned.csv", 'CsvFiles/movies_current_cleaned_2.csv') 


5021it [03:37, 23.08it/s]


In [ ]:
!pip install decord

     |████████████████████████████████| 13.6 MB 3.9 MB/s 


In [ ]:
import cv2
import math   
from decord import VideoReader
from decord import cpu, gpu

def frames_extraction_with_cut_in_middle2(video_path,cut_points_list,frame_rate,img_dim):
    # Empty List declared to store video frames
    frames_list = np.empty(shape=(len(cut_points_list),frame_rate,img_dim[0],img_dim[1],3),dtype = np.int)
    image_height,image_width = img_dim
    # Reading the Video File Using the VideoCapture
    vr = VideoReader(video_path, ctx=cpu(0),width=image_width, height=image_height,num_threads=12)
    # a file like object works as well, for in-memory decoding
    with open(video_path, 'rb') as f:
        vr = VideoReader(f, ctx=cpu(0),width=image_width, height=image_height,num_threads=6)

    length = int(len(vr))
    i = 0
    for start,end in cut_points_list:
        mid = int((end -start) /2)
        n_start = (mid+start) - int(frame_rate * frame_rate/2)
        n_end =  (mid+start) +int(frame_rate * frame_rate/2)
        rate = int(math.floor((n_end - n_start)/frame_rate))
        #print("start " , start , ", end ", end, " , mid ", mid ,", n_start ",n_start ,", n_end ", n_end )
        #print(rate)
        if frame_rate == 1:
            frames_list[i,] = vr[start + mid].asnumpy()
        else:
            if rate < 1 : 
                print("problem.")
                rate=1
            
            else:
                valid_nums = filter(lambda x: x % rate == 0, range(n_start, n_end))
                filtered_numbers = list(valid_nums)
                try:
                    if len(filtered_numbers) > frame_rate:
                        frames_list[i,] = vr.get_batch(filtered_numbers[len(filtered_numbers)-frame_rate:]).asnumpy()
                        
                    else:
                        frames_list[i,] = vr.get_batch(filtered_numbers).asnumpy()
                        
                except:
                        pass
        i = i + 1
    # returning the frames list 
    return frames_list


In [ ]:
from tensorflow import keras
import tensorflow as tf
from google.colab.patches import cv2_imshow
def preprocess_cutlists(video_path,in_path,out_path,model_path):
    class_names=['WritingN', 'WritingY']
    model = keras.models.load_model(model_path)
    df = pd.read_csv(in_path)
    list_of_datas = df['ID_Frame'].tolist()
    #print(list_of_datas)
    try:
        csv_out = pd.read_csv(out_path)
    except:  

        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points','Number_Scenes'])
        dataframe.to_csv(out_path)
        csv_out = pd.read_csv(out_path)
    skip_datas = csv_out['ID_Frame'].tolist()

    for data in tqdm(list_of_datas):
        if data in skip_datas:
            #print("Skipping the data ",data)
            pass
        else:
            if load_label(df,int(data)):
                #print("Working on the data ",data)
                cut_list =  load_frames_list(df,int(data))
                scenes = frames_extraction_with_cut_in_middle2(video_path+'/'+str(data)+'.mp4',cut_list,1,(224,224))
                
                new_cut_list = []
                for i,scene in enumerate(scenes): 
                    mid_frame = scene[0]
                    #print(mid_frame.shape)
                    
                    mid_frame=np.expand_dims(mid_frame, axis=0)
                    predictions = model.predict(mid_frame)
                    score = tf.nn.sigmoid(predictions[0])
                    index = np.argmax(score)
                    predicted = class_names[index]
                    
                   
                    if predictions[0] > 0.65:
                        #cv2_imshow( scene[0])
                        #print("I will DELETE. Score : ", predictions[0])
                        #show_model_layers(model, mid_frame)
                        pass
                    else: 
                        #cv2_imshow( scene[0])
                        #print("I will KEEP Score: ",predictions[0])
                        #show_model_layers(model, mid_frame)
                        new_cut_list.append(cut_list[i])
                new_row = {
                'ID_Frame':int(data),
                'Genres': load_label(df,int(data)),
                'Cut_Points': new_cut_list,
                'Number_Scenes': len(new_cut_list)
                }
                csv_out = csv_out.append(new_row, ignore_index=True)
                csv_out.to_csv(out_path,index=False)
            else: 
                  print('no label found for the data: ' , data)
        

In [ ]:
video_path='Data/'
#os.remove( 'CsvFiles/movies_current_final.csv')
model_path= 'WritingsDetector/writings_model_new.h5'
preprocess_cutlists(video_path, 'CsvFiles/movies_current_cleaned_2.csv', 'CsvFiles/movies_current_final.csv',model_path) 



100%|██████████| 5021/5021 [00:00<00:00, 9850.84it/s] 


In [ ]:
os.remove( 'CsvFiles/movies_current_dataset.csv')
clean_cut_points2("CsvFiles/movies_current_final.csv", 'CsvFiles/movies_current_dataset.csv') 


5021it [03:55, 21.32it/s]


#Change Labels to original labels

In [ ]:
def orginal_labels(orginal_csv, produced_csv_path, label_change_csv):
    df= pd.read_csv(label_change_csv)
    ids = df['movieId']
    try:
        csv_out = pd.read_csv(produced_csv_path)
    except:  

        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points','Number_Scenes'])
        dataframe.to_csv(produced_csv_path)
        csv_out = pd.read_csv(produced_csv_path)
    skip_datas = csv_out['ID_Frame'].tolist()

    df_org = pd.read_csv(orginal_csv)
    list_of_datas = df_org['ID_Frame'].tolist()
    for data in tqdm(list_of_datas):
        if data in skip_datas:
            #print("Skipping the data ",data)
            pass
        else:

            genres_str = (df.loc[ids == data])['genres'].values.tolist()
            if genres_str:
                genres = genres_str[0].split('|')
            new_cut_list = load_frames_list(df_org,data)
            #print(new_cut_list)
            
            new_row = {
               'ID_Frame':int(data),
               'Genres': genres,
               'Cut_Points': new_cut_list,
               'Number_Scenes': len(new_cut_list)
              }
            csv_out = csv_out.append(new_row, ignore_index=True)
            csv_out.to_csv(produced_csv_path,index=False)

In [ ]:
orginal_labels('CsvFiles/movies_current_dataset.csv', 'CsvFiles/movies_current_dataset_animation.csv','CsvFiles/movies.csv')

100%|██████████| 4694/4694 [00:05<00:00, 854.07it/s]  


#Remove a label


In [1]:
def remove_label(label_to_replace,csv_in,produced_csv_path):
    download_path = "Data/"
    try:
        csv_out = pd.read_csv(produced_csv_path)
    except:  

        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres','Cut_Points','Number_Scenes'])
        dataframe.to_csv(produced_csv_path)
        csv_out = pd.read_csv(produced_csv_path)
    skip_datas = csv_out['ID_Frame'].tolist()
    df_org= pd.read_csv(csv_in)
    movie_ids = df_org['ID_Frame']
    skip_datas = csv_out['ID_Frame'].tolist()

    for i in tqdm(movie_ids):
        if i in skip_datas:
            #print("Skipping the data ",data)
            pass
        #print(i)
        else:
            labels = load_label(df_org,i)
          
            if label_to_replace in labels:
                
                labels = [l for l in labels if not (l == label_to_replace)]
                
            elif labels:
               
                new_cut_list = load_frames_list(df_org,i)
                new_row = {
                  'ID_Frame':int(i),
                  'Genres': labels,
                  'Cut_Points': new_cut_list,
                  'Number_Scenes': len(new_cut_list)
                  }
                csv_out = csv_out.append(new_row, ignore_index=True)
                csv_out.to_csv(produced_csv_path,index=False)


In [ ]:
os.remove('CsvFiles/movies_current_dataset2.csv')
remove_label('Documentary','CsvFiles/movies_current_dataset.csv', 'CsvFiles/movies_current_dataset2.csv')

100%|██████████| 4804/4804 [03:28<00:00, 22.99it/s]
